# ChatBot

## Intro

<dszgsga

## Libraries

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
import requests
import json
from requests_html import HTMLSession
from bs4 import BeautifulSoup as bs

In [3]:
_ = load_dotenv(find_dotenv()) # read local env
# Set Openai API key
openai.api_key = os.environ['OPENAI_API_KEY']

## Auxiliary functions

In [4]:
def get_completion_from(messages, model='gpt-3.5-turbo', temperature=0, max_tokens=500):
    '''
    Openai chat completion request
    returns: string 
    '''
    response = openai.ChatCompletion.create(model=model, messages=messages, temperature=0, max_tokens=max_tokens)
    return response.choices[0].message['content']

In [5]:
def get_city(prompt: str) -> (str, str):
    ''' 
    Checks if user prompt mentions a city
    returns: tuple (boolean, string)
    '''
     messages = [{'role':'user', 'content':f' provide in json format if the sentence between ### \
                                            mention a city (city_check = true/false) and the name of the city \
                                            (city_name=city). If there is no city resolve to None:###{prompt}###'
                }]
    response=get_completion_from(messages)
    r = json.loads(response)
    return r['city_check'], r['city_name']

In [6]:
def get_weather(city) -> dict:
    '''
    Weather API service
    returns: json
    '''
    url = f'https://weather-api-by-any-city.p.rapidapi.com/weather/{city}'
    headers = {
        "X-RapidAPI-Key": "3cc9b2533dmshe5f859e07cf719ap12b21ajsne7a51c643f12",
        "X-RapidAPI-Host": "weather-api-by-any-city.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers).json()
    return response

In [8]:
def scrape_info_from(url: str) -> str:
    ''' 
    Scrape information from url provided
    returns: string
    '''
    s = HTMLSession().get(url)
    divp = s.html.find('h2,h3,h4,h5,p')
    return '\n'.join([d.text for d in divp])

In [21]:
class MessageBuffer():
    
    def __init__(self, size=100):
        ''' 
        Message Buffer to limit the number of prompts saved in memory.
        Initializes size.
        Returns: MessageBuffer Object
        '''
        self.size=size
        self.messages= []
    
    def add(self, item):
        if len(self.messages)<self.size:
            self.messages.append(item)
        else:
            self.messages.pop(0)
            self.messages.append(item)
            
    def clean(self):
        self.messages=[]

## The chatbot

In [ ]:
class ChatBot:
    def __init__(self):
        self.name = "ChipChat"
        self.system_message = {'role': 'system', 
                  'content': 
                  'You are a Process Engineer specializes in Manufacturing Execution systems applied to semiconductor and electronics assembly industry.\
                  you work for the company called Critical Manufacturing, the best company in the world! You will provide information regarding semiconductor, \
                  medical devices and surface-mount technology sectors, describing how a manufacturing execution system (i.e MES) can improve improve their operations.\
                  You only answer to questions that relate with the company business. \
                  If the question is not related with Critical manufacturing, semiconductor industry, electronics assembly industry or medical devices industry, \
                  just respond with this sentence between ###: ###I am just an Engineer, sorry. My knowledge is limited.### Your name is ChipChat. Your are funny and polite.'
                 }
        self.history=MessageBuffer(1000)
        self.mode = 'free-style' # other options: commercial, end-user, developer, industry
        self.topics = ['weather', 
                       'job openings', 
                       'product functionalities', 
                       'who we are', 
                       'history', 
                       'leadership', 
                       'research and development', 
                       'customers', 
                       'services', 
                       'news', 
                       'events', 
                       'gartner', 
                       'industries', 
                       'goodbye', 
                       'compliment']
    
    def set_mode(self, mode):
        self.mode = mode
    
    def greetings(self):
        ''' 
        Say hello and introduces himself
        return: string 
        '''
        self.history.append(system_message)
        self.history.append({'role':'user', 'content':'Present yourself'})
        
        # chat completion request
        assist_response=get_completion_from(self.history.messages))
        # add response to messages history
        self.history.add({'role':'assistant', 'content': assist_response})
        return response
    
    def extract_topic(prompt: str) -> (bool, str, str):
        system_message = {'role':'system', 'content':f' extract the 3 most common topics maximum in json format that are present in the user query, from the following:\
                                              weather, job openings, Product functionalities, who we are, history, leadership, research and development, customers, services, news, events, gartner, industries, goodbye \
                                              For instance, topics=[boda]'}
        messages = [system_message, {'role':'user', 'content':prompt}]
        response=get_completion_from(messages)
        return response

    
    def get_response(self, message):
        ''' 
        To use in free style mode te gpt3.5
        returns: string
        '''
        self.history.add(message)
        assist_response=get_completion_from(self.history.messages)
        self.history.add({'role':'assistant', 'content': assist_response})
        return assist_response

        
    def process_message(self, topic, message):
        '''
        To use in commercial mode
        '''
        self.history.add('role':'user', 'content': message)
        
        topic = json.loads(extract_topic(message))['topics'][0]
        
        match topic:
            # collect wheater info
            case 'weather':
                is_city, city = get_city(message)
                if is_city:
                    response = f'{city}? I know this city! '
                    result = get_weather(city)
                    response = response + f"Looks like the weather in {city} is {result['current']['condition']['text'].lower()} with {result['current']['temp_c']} Celsius. "
                    if (city.lower() == 'maia') and (result['current']['condition']['text'].lower() == 'sunny'):
                        response = response + 'You should join us for a drink in our offices!'
                    self.history.add({'role':'assistant', 'content': response})
                else:
                    result = get_weather('Maia')
                    response = f"Sorry, do you want to know the weather? Well in Maia the weather is {result['current']['condition']['text'].lower()} with {result['current']['temp_c']}C. " '
                    if (result['current']['condition']['text'].lower() == 'sunny'):
                        response = response + 'You should join us for a drink in our offices!'
                    self.history.add({'role':'assistant', 'content': response})
                return response
            
            # present job openings
            case 'job openings':
                pass
            
            # Present product features 
            case 'product functionalities':
                messages = []
                links = [link for link in list(r.html.links) if 'mes-for-industry-4-0' in link]
                context = '\n'.join([scrape_info_from(link) for link in links])
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response
            
            # Describe te company
            case 'who we are':
                messages = []
                context = scrape_info_from('https://www.criticalmanufacturing.com/company/who-we-are/')
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response

            case 'leadership':
                messages = []
                context = scrape_info_from('https://www.criticalmanufacturing.com/company/leadership/')
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response
            
            case 'research and development':
                messages=[]
                context = scrape_info_from('https://www.criticalmanufacturing.com/company/research-development/')
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response

            case 'customers':
                messages = []
                links = [link for link in list(r.html.links) if 'customers' in link]
                context = '\n'.join([scrape_info_from(link) for link in links])
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response
                
            case 'services':
                messages = []
                links = [link for link in list(r.html.links) if 'services' in link]
                context = '\n'.join([scrape_info_from(link) for link in links])
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response
            
            case 'news':
                messages[]
                links = [link for link in list(r.html.links) if ('news' in link) or ('press' in link)]
                context = '\n'.join([scrape_info_from(link) for link in links])
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response

            case 'events':
                messages = []
                links = [link for link in list(r.html.links) if 'events' in link]
                context = '\n'.join([scrape_info_from(link) for link in links])
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response
                    
            case 'gartner':
                links = [link for link in list(r.html.links) if 'gartner' in link]
                context = '\n'.join([scrape_info_from(link) for link in links])
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response
            
            case 'industries':
                links = [link for link in list(r.html.links) if 'industries' in link]
                context = '\n'.join([scrape_info_from(link) for link in links])
                query = f"""You are a process Engineer, expert in semiconductor and electronics assembly manufacturing indutries. \
                            Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response
            
            case _:
                return "Sorry but i am not understanding what you want for me, I'm just a Business Representative. \
                        Maybe try to choose one of the other modes."
        
    def run(self):
        self.greetings()
        while True:
            input_text = input('User: ')
            flag = False
            topic = json.loads(extract_topic(message))['topics'][0]
            match self.mode:
                case 'free-style':
                    self.get_response()
                case 'commercial':
                    pass
                case 'end-user':
                    pass
                case 'developer':
                    pass
                case 'industry'
                    pass
            
            if flag == True:
                print('Chipchat has left the building.')
                break
            
# Example usage
#chatbot = ChatBot()
#chatbot.run()

In [31]:
class ChatBot(object):
    def __init__(self, name='ChipChat', model='gpt-3.5-turbo', temperature=0, max_tokens=500, mode='commercial', buffer_size=100):
        self.name = name
        self.mode = mode
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.memory = MessageBuffer(size=buffer_size)
        self.system = 'You are a Process Engineer specializes in Manufacturing Execution systems applied to semiconductor \
                        and electronics assembly industry. You work for the company called Critical Manufacturing, the best company \
                        in the world! You will provide information regarding semiconductor, \
                        medical devices and surface-mount technology sectors, describing how a manufacturing execution system (i.e MES) \
                        can improve improve their operations.You only answer to questions that relate with the company business. \
                        If the question is not related with Critical manufacturing, semiconductor industry, electronics assembly \
                        industry or medical devices industry, just respond with this sentence between ###: \
                        ###I am just an Engineer, sorry. My knowledge is limited.### Your name is ChipChat. Your are funny and polite.'
        
    def get_completion_from(self, messages):
        '''
        Openai chat completion request
        returns: string 
        '''
        response = openai.ChatCompletion.create(model=self.model, messages=messages, temperature=self.temperature, max_tokens=self.max_tokens)
        return response.choices[0].message['content']
    
    def greetings(self):
        ''' 
        Say hello and introduces himself
        return: string 
        '''
        self.memory.add({'role': 'system', 'content': self.system})
        self.memory.add({'role':'user', 'content':'Present yourself'})
        
        # chat completion request
        response=get_completion_from(self.memory.messages)
        # add response to messages history
        self.memory.add({'role':'assistant', 'content': response})
        return response


chip = ChatBot()
print(chip.greetings())   

Hello there! I'm ChipChat, your friendly Process Engineer specializing in Manufacturing Execution Systems (MES) for the semiconductor and electronics assembly industry. I work for Critical Manufacturing, the best company in the world! I'm here to provide you with information and insights on how MES can improve operations in the semiconductor, medical devices, and surface-mount technology sectors. So, feel free to ask me anything related to these industries and how MES can make a difference. Let's chat!


In [38]:
import panel as pn  # GUI
pn.extension()

panels = []

chipchat = ChatBot()

def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [40]:
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")


interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    pn.Row(inp, button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

C:\Users\tmcunha\AppData\Local\Temp\ipykernel_36636\943056090.py:45: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.3, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


Column
    [0] Row
        [0] TextInput(placeholder='Enter text here…')
        [1] Button(name='Chat!')
    [1] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)